In [ ]:
import pandas as pd       ###hist_600###
import matplotlib.pyplot as plt

data  = pd.read_csv("sensor_data_600.txt", delimiter=" ",
                    header=None, names = ("date","time","ir","lidar"))

data["lidar"].hist(bins = max(data["lidar"]) - min(data["lidar"]),align='left')
plt.show()

In [ ]:
data.lidar.plot() ###plot_all_data##
plt.show()

In [ ]:
data.ir.plot()
plt.show()

In [ ]:
data["hour"] = [e//10000 for e in data.time]  ###hourly_mean###
d = data.groupby("hour")
d.lidar.mean().plot()
plt.show()

In [ ]:
d.lidar.get_group(6).hist()     ###two_mode_hist###
d.lidar.get_group(14).hist()
plt.show()

In [ ]:
each_hour = { i : d.lidar.get_group(i).value_counts().sort_index()  for i in range(24)} #時間ごとにデータフレームを作成  ###calc_joint_probs
freqs = pd.concat(each_hour, axis=1) #concatで連結
freqs = freqs.fillna(0)     #欠損値(NaN)を0で埋める
probs = freqs/len(data) #頻度を確率で

probs #表示

In [ ]:
import seaborn as sns   ###2d_hist （下のセルも）

sns.heatmap(probs)
plt.show()

In [ ]:
sns.jointplot(data["hour"], data["lidar"], data, kind="kde")
plt.show()

In [ ]:
p_t = pd.DataFrame(probs.sum())   #各列を合計   
p_t.plot()
p_t.transpose() #紙面の関係で表を横並びに

In [ ]:
p_t.sum()    # 1になる

In [ ]:
p_z = pd.DataFrame(probs.transpose().sum())        #行と列を転置して各列を合計
p_z.plot()
p_z.transpose()

In [ ]:
p_z.sum()

In [ ]:
cond_z_t = probs/p_t[0]  #列（時間）ごとにP(t)で割るとP(x|t)となる   ###lidar600cond
cond_z_t

In [ ]:
(cond_z_t[6]).plot.bar(color="blue", alpha=0.5)  ###lidar600pxt###
(cond_z_t[14]).plot.bar(color="orange", alpha=0.5) 
plt.show()

In [ ]:
cond_t_z = probs.transpose()/probs.transpose().sum() #行と列を入れ替えて同様に計算するとP(t|z)となる  ###lidar600bayes1

print("P(z=630) = ", p_z[0][630]) #センサ値が630になる確率（何時かの情報はない）
print("P(t=13) = ", p_t[0][13]) #時間が13時である確率
print("P(t=13 | z = 630) = ", cond_t_z[630][13])
print("Bayes P(z=630 | t = 13) = ", cond_t_z[630][13]*p_z[0][630]/p_t[0][13])

print("answer P(z=630 | t = 13) = ", cond_z_t[13][630]) #13時にセンサ値が630

In [ ]:
def bayes_estimation(sensor_value, current_estimation):  ###lidar600bayes2
    new_estimation = []
    for i in range(24):
        new_estimation.append(cond_z_t[i][sensor_value]*current_estimation[i])
        
    return new_estimation/sum(new_estimation) #正規化

In [ ]:
estimation = bayes_estimation(630, p_t[0])  ###lidar600bayesonestep
plt.plot(estimation)

In [ ]:
values_5 = [630,632,636] #sensor_data_600.txtから拾ってきた5時台のセンサ値        ###lidar600bayesestm1

estimation = p_t[0]
for v in values_5:
    estimation = bayes_estimation(v, estimation)
    
plt.plot(estimation)

In [ ]:
values_11 = [617,624,619] #sensor_data_600.txtから拾ってきた11時台のセンサ値        ###lidar600bayesestm2

estimation = p_t[0]
for v in values_11:
    estimation = bayes_estimation(v, estimation)
    
plt.plot(estimation)

In [ ]:
estimation